In [99]:
import json
f = open("labels_v2.json")
label = json.load(f)
#ruled_based matcher
import spacy
#nlp = spacy.load("/Users/luis/Documents/GitHub/2022Summer/AI-Oriented-Recommendation-System/PreprocessingData/spaCy/rule_model_7Categories")

In [100]:
nlp = spacy.load("en_core_web_lg")
import nltk 

In [102]:
from spacy import displacy
doc = nlp("long sleeve shirt")
displacy.render(doc)

In [18]:
nltk.pos_tag(("graphic raverback tank").split(" "))

[('graphic', 'JJ'), ('raverback', 'NN'), ('tank', 'NN')]

In [19]:
pt = label.get("product_type")
st = label.get("sub_product_type")

In [20]:
for key in st[0].keys():
    print(key)

sandals
sneakers
heels
boots
shirts
sweaters/hoodies
jackets/coats
activewear
shorts
skirts
pants
jeans
dresses
swimwear
jewelries
bags
glasses
headwear
cosmetics
skin care
beauty tools
hair care
nail care


In [21]:
#8 categories
shoes = set()
tops = set()
bottoms = set()
other_clothing = set()

beauty = set()
accessories = set()
homeware = set()
other = set()

In [97]:
#method to compare two string similarities
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [124]:
similar("one-piece", "onesie")

0.6666666666666666

In [125]:
token1 = nlp("one piece")
token2 = nlp("onesie")
token1.similarity(token2)

0.15814635741847283

In [127]:
from typing import List
def check_and_concat_punct(temp:List[str]) -> str:
    '''
    input: list, e.g. ['t', '-', 'shirt'], ['high', '-', 'rise', 'shoe']
    function: deal with the list with '-' punct
    output: "t-shirt" "high-rise shoe" "a-b-c color" 
    '''
    temp_string = ""
    if temp.__contains__('-') and len(temp) == 3:
        temp_string = ''.join(temp)
    elif temp.__contains__('-') and len(temp) > 3:
        
        pos = temp.index('-')
        #if there is wrong position of '-'
        if pos < 1:
            temp_string = ' '.join(temp)
            return temp_string
        
        puct_string = temp[pos - 1] + temp[pos] + temp[pos + 1]
        rest_string = ' '.join(temp[pos + 2:])
        temp_string = puct_string + ' '+ rest_string
    else:
        temp_string = ' '.join(temp)
    
    return temp_string 
    

In [126]:
'''
input: original string e.g. 'scandals', 'jackets/coats'ArithmeticError
funct: clean the string to the original words and put to list
output: ['scandal'], ['jacket', 'coats'] 
'''
def lemma_string(original_string:str) -> List:
    doc = nlp(original_string.lower())
    temp = []
    for token in doc:
        if token.text == '&' or token.text == 'and' or token.text == '/' or token.text == ',':
            continue
        if token.text == "glasses" or token.text == 'booty' or token.text == "sunglasses" or token.text == "earrings": #some of them should be represented in plural format
            temp.append(token.text)
        else:
            temp.append(token.lemma_)
    return temp


In [27]:
lemma_string("Jumpsuits & Rompers")

['jumpsuit', 'romper']

In [128]:
sub_cat_all_keys = [key for key in st[0].keys()]
sub_cat_shoes_keys = sub_cat_all_keys[0:4]
sub_cat_tops_keys= sub_cat_all_keys[4:7]
sub_cat_other_clothing_keys = sub_cat_all_keys[7:8] + sub_cat_all_keys[13:14]
sub_cat_bottoms_keys = sub_cat_all_keys[8:13]
sub_cat_acc_keys = sub_cat_all_keys[14: 19]
sub_cat_beauty_keys = sub_cat_all_keys[19: ]
print(sub_cat_shoes_keys)
print(sub_cat_tops_keys)
print(sub_cat_other_clothing_keys)
print(sub_cat_bottoms_keys)
print(sub_cat_acc_keys)
print(sub_cat_beauty_keys)



['sandals', 'sneakers', 'heels', 'boots']
['shirts', 'sweaters/hoodies', 'jackets/coats']
['activewear', 'swimwear']
['shorts', 'skirts', 'pants', 'jeans', 'dresses']
['jewelries', 'bags', 'glasses', 'headwear', 'cosmetics']
['skin care', 'beauty tools', 'hair care', 'nail care']


In [129]:
#This map the product name to main_7categories type
#will be like {"slide" : 1} 1 is mapping to shoes
#from nltk.stem import PorterStemmer
#lemmatizer = PorterStemmer() #since this is not accurate as spacy.lemmanizer, don't use it.
all_main = {key for key in pt[0].keys()}
print("First: \n",  all_main)
#set
all_sub_main = set()
#dict { 'shoes': {(all shoe types whatever it is the 2nd or 3rd class label)} }
products_to_all = {}
#dict { 'shoes': 1, ''}
main_categories_map_to_num = {}
#dict { 'heels': 1, 't-shirt': } specific item maps to the number
specific_products_map_to_num = {}
pt_index = 0
for key in pt[0].keys():
    product_name_2nd_list = pt[0].get(key)
    pt_key = key
    pt_value_set = set()
    
    for item_string in product_name_2nd_list:
        temp = lemma_string(item_string)
        temp_string = check_and_concat_punct(temp)

        #update set
        all_sub_main.add(temp_string)
        #update the value in the pair[]
        pt_value_set.add(temp_string)
    
    #main_to_sub map
    products_to_all.update({pt_key:pt_value_set})
    #product_to_num map
    pt_index += 1
    main_categories_map_to_num.update({pt_key:pt_index})

main_categories_map_to_num.update({"other" : 8})
main_categories_map_to_num.update({"unknown" : 0})
#if key in [scandals, sneakers, heels, boots], or [boots, shirts, sweaters, jackets, coats]
#
print("Second: all main_categories\n", all_sub_main)

all_sub = set()
for key in st[0].keys():
    product_name_3rd_list = st[0].get(key) # get list of []
    for item_string in product_name_3rd_list: 
        #loop all item_string in order
        temp = lemma_string(item_string)
        temp_string = check_and_concat_punct(temp)
        temp_list = lemma_string(key)
        temp_string2 = check_and_concat_punct(temp_list)
        all_sub.add(temp_string)
        all_sub.add(temp_string2)
print("Third: all sub catrgories\n", all_sub)


First: 
 {'homeware', 'shoes', 'tops', 'other', 'beauty', 'bottoms', 'accessories', 'other_clothing'}
Second: all main_categories
 {'textile', 'sweater', 'perfume', 'jewelry', 'mask', 'soap', 'tank top', 'artwork', 'sandal', 'skateboard wheel', 'underwear', 'food', 'sunglasses', 'overall', 'wakesurfs', 'nail care', 'bath robe', 'sticker', 'duvet cover', 'intimate', 'headband', 'tee shirt', 'athletic boxer', 'cologne', 'coat', 'raverback tank', 'bandana', 'card stationery', 'wallet', 'scarf', 'tee', 'knitwear', 'boot', 'blouse', 'heel cover', 't-shirt', 'flat shoe', 'vest', 'sweatshirt', 'romper', 'clogs', 'skateboard wax', 'waistcoat', 'phone case', 'bath mat', 'jean', 'short', 'pant', 'ceramic', 'culotte', 'croptee', 'clip', 'sweatpant', 'jersey', 'sleepwear', 'shoe lace', 'puzzle', 'jumpsuit romper', 'dress', 'backpack', 'wedge', 'shower curtain', 'collar', 'shirt', 'skate shoe', 'skateboard tool', 'clock', 'bralette', 'headwear', 'slipper', 'cosmetic', 'blanket', 'sneaker', 'bag', '

In [130]:
'''
input: the list of all sub_categories, the key of the main_categories
function:put all sub_categories to the relative the main_categories set
output: none
'''
def update_products_to_all(string_list : List, main_key: str)->None:
    assert len(string_list) > 0
    assert products_to_all.get(main_key) is not None
    for item_string in string_list:
        
        temp = lemma_string(item_string)
        temp_string = check_and_concat_punct(temp) 
        products_to_all.get(main_key).add(temp_string)


In [131]:
products_to_all

{'shoes': {'boot',
  'clogs',
  'flat shoe',
  'heel',
  'oxford loafer',
  'sandal',
  'skate shoe',
  'slipper',
  'sneaker',
  'snowshoe',
  'wedge'},
 'tops': {'blazer',
  'blouse',
  'bra',
  'bralette',
  'camisole',
  'cardigan',
  'coat',
  'croptee',
  'croptop',
  'hoodie',
  'intimate',
  'jacket',
  'jersey',
  'kimono',
  'knitwear',
  'poncho',
  'raverback tank',
  'shirt',
  'sweater',
  'sweatshirt',
  't',
  't-shirt',
  'tank',
  'tank top',
  'tee',
  'tee shirt',
  'vest',
  'waistcoat'},
 'bottoms': {'athletic boxer',
  'culotte',
  'jean',
  'jogger',
  'legging',
  'pant',
  'short',
  'skirt',
  'sweatpant',
  'trouser'},
 'other_clothing': {'activewear',
  'cloak',
  'dress',
  'jumpsuit',
  'jumpsuit romper',
  'overall',
  'romper',
  'sleepwear',
  'swimwear',
  'underwear'},
 'beauty': {'beauty tool',
  'cosmetic',
  'hair care',
  'nail care',
  'perfume',
  'skin care'},
 'accessories': {'backpack',
  'bag',
  'bandana',
  'belt',
  'clip',
  'collar',
 

In [132]:
main_categories_map_to_num

{'shoes': 1,
 'tops': 2,
 'bottoms': 3,
 'other_clothing': 4,
 'beauty': 5,
 'accessories': 6,
 'homeware': 7,
 'other': 8,
 'unknown': 0}

In [33]:
#add all sub categories to the main 8 categories.
for key in sub_cat_all_keys:#loop all keys in sub_main_categories, 
    if key in sub_cat_tops_keys:#if the key belongs to tops
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_tops_keys, 'tops')
    elif key in sub_cat_shoes_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_shoes_keys, 'shoes')
    elif key in sub_cat_bottoms_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_bottoms_keys, 'bottoms')
    elif key in sub_cat_other_clothing_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_other_clothing_keys, 'other_clothing')
    elif key in sub_cat_acc_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_acc_keys, 'accessories')
    elif key in sub_cat_beauty_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_beauty_keys, 'beauty') 
    

In [34]:
products_to_all

{'shoes': {'ankle boot',
  'ankle strap heel',
  'boot',
  'canvas sneaker',
  'chelsea boot',
  'chunky sandal',
  'classic sneaker',
  'clog',
  'espadrille',
  'falt sandal',
  'flat shoe',
  'flip flop',
  'heel',
  'heel boot',
  'heel wedge',
  'high top sneaker',
  'ice gripper boot',
  'jelly',
  'kitten heel',
  'lace-up boot',
  'mule',
  'oxford loafer',
  'platform',
  'platform boot',
  'platform heel',
  'pump',
  'rain boot',
  'sandal',
  'skate shoe',
  'slide',
  'slip-on sneaker',
  'slipper',
  'sneaker',
  'snowshoe',
  'stilettos',
  'tall boot',
  'waterproof boot',
  'wedge',
  'wedge heel',
  'winter boot'},
 'tops': {'blazer',
  'blouse',
  'bomber jacket',
  'bra',
  'bralette',
  'button-down shirt',
  'button-up shirt',
  'camisole',
  'cardigan',
  'coat',
  'crop hoodie',
  'crop shirt',
  'cropped sweater',
  'croptee',
  'croptop',
  'denim jacket',
  'dress shirt',
  'drop tee',
  'graphic tee',
  'hoodie',
  'intimate',
  'jacket',
  'jacket coat',
  

In [35]:
main_categories_map_to_num

{'shoes': 1,
 'tops': 2,
 'bottoms': 3,
 'other_clothing': 4,
 'beauty': 5,
 'accessories': 6,
 'homeware': 7,
 'other': 8,
 'unknown': 0}

In [36]:
for key in products_to_all.keys():
    products_to_all.update({key: list(products_to_all.get(key))})

import json

with open("product_to_all.json", "w") as outfile:
    json.dump(products_to_all, outfile)
#parsed = json.dumps(products_to_all, indent=4)
with open("main_categories_to_num.json","w") as f:
    json.dump(main_categories_map_to_num,f)



In [37]:
f = open("main_categories_to_num.json")
map_num =json.load(f)
map_num

{'shoes': 1,
 'tops': 2,
 'bottoms': 3,
 'other_clothing': 4,
 'beauty': 5,
 'accessories': 6,
 'homeware': 7,
 'other': 8,
 'unknown': 0}

In [38]:
f = open("product_to_all.json")
parsed = json.load(f)
parsed.keys()

dict_keys(['shoes', 'tops', 'bottoms', 'other_clothing', 'beauty', 'accessories', 'homeware', 'other'])

In [42]:
'''
Build the hashmap all products has mapped to the number of categories
'''
for key in products_to_all.keys():
    map_number_of_this_key = main_categories_map_to_num.get(key)
    assert products_to_all.get(key) is not None
    #include the main_categories to number;
    specific_products_map_to_num.update({key: map_number_of_this_key})
    for item in products_to_all.get(key):
        specific_products_map_to_num.update({item: map_number_of_this_key})
len(specific_products_map_to_num)


332

In [43]:
specific_products_map_to_num

{'shoes': 1,
 'oxford loafer': 1,
 'wedge': 1,
 'pump': 1,
 'winter boot': 1,
 'boot': 1,
 'skate shoe': 1,
 'ankle boot': 1,
 'slide': 1,
 'ankle strap heel': 1,
 'sandal': 1,
 'wedge heel': 1,
 'lace-up boot': 1,
 'falt sandal': 1,
 'snowshoe': 1,
 'ice gripper boot': 1,
 'slipper': 1,
 'flat shoe': 1,
 'stilettos': 1,
 'sneaker': 1,
 'rain boot': 1,
 'flip flop': 1,
 'high top sneaker': 1,
 'platform heel': 1,
 'chelsea boot': 1,
 'heel wedge': 1,
 'clog': 1,
 'jelly': 1,
 'canvas sneaker': 1,
 'mule': 1,
 'espadrille': 1,
 'chunky sandal': 1,
 'platform': 1,
 'heel': 1,
 'heel boot': 1,
 'kitten heel': 1,
 'classic sneaker': 1,
 'platform boot': 1,
 'tall boot': 1,
 'slip-on sneaker': 1,
 'waterproof boot': 1,
 'tops': 2,
 'poncho': 2,
 'sweater': 2,
 'technical sport jacket': 2,
 'tee': 2,
 'crop hoodie': 2,
 'button-up shirt': 2,
 'cropped sweater': 2,
 'knitwear': 2,
 'sweater hoodie': 2,
 'tank top': 2,
 'shirt': 2,
 'quilt down jacket': 2,
 'button-down shirt': 2,
 'short slee

In [44]:
all_cat = all_sub.union(all_sub_main).union(all_main)

In [45]:
#compare the products_to_all with all_cat
comp = set()
for val in products_to_all.values():
    comp = comp.union(val)

all_cat.difference(comp)

{'accessories',
 'beauty',
 'bottoms',
 'homeware',
 'other',
 'other_clothing',
 'shoes',
 'tops'}

In [46]:
len(all_cat)

332

In [48]:

for item in all_cat:
    if item not in specific_products_map_to_num.keys():
        print(item)


In [49]:
import pandas as pd
from string_grouper import match_strings, match_most_similar, \
    group_similar_strings, compute_pairwise_similarities


In [50]:
df = pd.read_csv("temp_not_finished.csv")
#df.insert(df.columns.get_loc('1st_class_label'), "product_type(modified)", '', allow_duplicates=True)
#df.insert(df.columns.get_loc('1st_class_label'), "match_most_similar_>80%_string", '', allow_duplicates=True)
#df.insert(df.columns.get_loc('1st_class_label'), "match_string_>60%_similarity", '', allow_duplicates=True)
#df.insert(df.columns.get_loc('1st_class_label'), "group_rep_product_type", '', allow_duplicates=True)
#df.rename(columns={"Unnamed: 0":"current_index", "index": "original_index"}, inplace=True)
df.columns

Index(['current_index', 'original_index', 'id', 'title', 'tags', 'body_html',
       'product_type', 'product_type(modified)',
       'match_most_similar_>80%_string', 'match_string_>60%_similarity',
       'group_rep_product_type', '1st_class_label', '2nd_class_label',
       '3rd_class_label', 'raw_text'],
      dtype='object')

In [51]:
#df.drop("Unnamed: 0", inplace=True, axis=1)

In [178]:
#df.to_csv("temp_not_finished.csv", index=False)

In [51]:
#remove pre_suffix of the original 'product_type' to match more accurate items
removable_words = ["mens", "womens", "girls", "ladies", "boys", "graphic", "premium", "cozy", "comfort", "casual", 'adult']
def modify_product_type(original:str)-> str:
    ori_list = original.split(" ")
    res = []
    for e in ori_list:
        if e in removable_words:
            continue
        else:
            res.append(e)
    return " ".join(res)

In [52]:
for i in range(df.shape[0]):
    ori_word = df.loc[i, ['product_type']].values[0]
    df.loc[i, ['product_type(modified)']] = modify_product_type(ori_word)

In [266]:
df.squeeze()

,current_index,original_index,id,title,tags,body_html,product_type,product_type(modified),match_most_similar_>80%_string,match_string_>60%_similarity,group_rep_product_type,1st_class_label,2nd_class_label,3rd_class_label,raw_text
0,0,1,6750552359068,Hues Underwire One Piece,"{d-cup+,meta-size-chart-artesands-size-guide,m...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",one piece,one piece,NaN,NaN,NaN,0,0,0,Hues Underwire One Piece; d cup+ meta size cha...
1,1,3,7552752550106,Marble Malibu Set,"{crop,meta-size-chart-bound-size-guide,moderat...","<p data-mce-fragment=""1""><span>A Bond-Eye orig...",bikini top,bikini top,NaN,NaN,NaN,0,0,0,Marble Malibu Set; crop meta size chart bound ...
2,2,4,5918371152028,Sophia Bottom,"{color-patterned,high-waist,meta-size-chart-po...","<meta charset=""utf-8"">\n<p>Meet your newest gi...",bikini bottom,bikini bottom,NaN,NaN,NaN,0,0,0,Sophia Bottom; color patterned high waist meta...
3,3,6,5918535942300,Sonja Top,"{color-stripes,crop,meta-size-chart-power-2-th...","<meta charset=""utf-8"">\n<p>This retro revival ...",bikini top,bikini top,NaN,NaN,NaN,0,0,0,Sonja Top; color stripes crop meta size chart ...
4,4,7,6613956362396,Shirley Tie-Side,"{high-waist,meta-size-chart-power-2-the-flower...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",bikini bottom,bikini bottom,NaN,NaN,NaN,0,0,0,Shirley Tie Side; high waist meta size chart p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17968,17968,22069,2482003738706,Dree Crop Hi Rise / Cameo,"{""Citizens of Humanity"",Denim}","<meta charset=""utf-8"">\n<p class=""product-deta...",pant,pant,NaN,NaN,NaN,0,0,0,Dree Crop Hi Rise / Cameo; Citizens of Humanit...
17969,17969,22070,2408859238482,Rako Hoops / Sterling Silver,"{Earrings,""Fay Andrada""}","<meta charset=""utf-8"">\n<p><span>Lightweight h...",jewelry,jewelry,NaN,NaN,NaN,0,0,0,Rako Hoops / Sterling Silver; Earrings Fay And...
17970,17970,22071,1487071740013,Jamie Hi Rise Classic Fit / Plume,"{AGOLDE,Bottoms,Denim}","<meta charset=""utf-8"">\n<p><span data-v-19a2cf...",pant,pant,NaN,NaN,NaN,0,0,0,Jamie Hi Rise Classic Fit / Plume; AGOLDE Bott...
17971,17971,22072,147355598868,Tait Earrings in Brass and Malachite,"{Earrings,Quarry}",<p><span>The Tait Earrings are detailed with a...,jewelry,jewelry,NaN,NaN,NaN,0,0,0,Tait Earrings in Brass and Malachite; Earrings...


In [53]:
pre_defiened_labels = pd.Series(list(all_cat), name="pre_defined_label")
most_similar_matches = match_most_similar( pre_defiened_labels, df["product_type(modified)"],\
    min_similarity = 0.80, ignore_index=False, replace_na=False)
most_similar_matches

,most_similar_index,most_similar_pre_defined_label
0,169.0,one piece
1,57.0,bikini top
2,204.0,bikini bottom
3,57.0,bikini top
4,204.0,bikini bottom
...,...,...
17968,73.0,pant
17969,168.0,jewelry
17970,73.0,pant
17971,168.0,jewelry


In [268]:
pre_defiened_labels

0      blue light block glasses
1               button-up shirt
2                        blouse
3                        collar
4                       slipper
                 ...           
327                     sneaker
328                      beauty
329                         bag
330                     shampoo
331                   tall boot
Name: pre_defined_label, Length: 332, dtype: object

In [54]:
most_similar_matches = pd.concat([df["original_index"],df["product_type(modified)"], most_similar_matches], axis=1)
most_similar_matches.to_csv("(GS)most_similar_string.csv")
most_similar_matches.shape


(17973, 4)

In [55]:
matches = match_strings(pre_defiened_labels, df["product_type(modified)"],\
    min_similarity = 0.65, ignore_index = False, replace_na=False)
matches.to_csv("(GS)match_strings.csv")
matches.shape

(13043, 5)

In [56]:
grouped_data = group_similar_strings(
    df["product_type(modified)"],
    min_similarity=0.60,
    ignore_index = False,
    replace_na = False
)
grouped_data.to_csv("(GS)group_similar_strings.csv")
grouped_data.shape

(17973, 2)

### Build 4 categories label dictionary


In [91]:
#accessory, other, 
all_cat

{'accessories',
 'activewear',
 'air freshener',
 'ankle boot',
 'ankle strap heel',
 'anklet',
 'area rug',
 'artwork',
 'athletic boxer',
 'aviator glasses',
 'backpack',
 'bag',
 'baggy jean',
 'bandana',
 'base layer',
 'baseball hat',
 'bath bomb',
 'bath mat',
 'bath robe',
 'beauty',
 'beauty tool',
 'belt',
 'bikini bottom',
 'bikini top',
 'blanket',
 'blazer',
 'blouse',
 'blue light block glasses',
 'blush',
 'board short',
 'body wash shower gel',
 'bodycon dress',
 'bomber jacket',
 'book',
 'boot',
 'bootcut jean',
 'booty short',
 'bottle',
 'bottoms',
 'bra',
 'bracelet',
 'bralette',
 'bronzer',
 'brooch',
 'brush cleaner',
 'brush set',
 'button-down shirt',
 'button-up shirt',
 'camisole',
 'candle',
 'canteen',
 'canvas',
 'canvas sneaker',
 'cap',
 'capri',
 'card stationery',
 'cardigan',
 'cargo short',
 'cargos pant',
 'cat eye glasses',
 'ceramic',
 'chelsea boot',
 'chinos pant',
 'chunky sandal',
 'classic sneaker',
 'cleanser',
 'clip',
 'cloak',
 'clock',
 

In [92]:
all_cat.__contains__("jumpsuit romper")

True

In [93]:
similar("acc", "accessories")

0.42857142857142855

In [37]:
print(most_similar_matches.columns)
print(matches.columns)
print(grouped_data.columns)

Index(['original_index', 'product_type(modified)', 'most_similar_index',
       'most_similar_pre_defined_label'],
      dtype='object')
Index(['left_index', 'left_pre_defined_label', 'similarity',
       'right_product_type(modified)', 'right_index'],
      dtype='object')
Index(['group_rep_index', 'group_rep_product_type(modified)'], dtype='object')


In [57]:
most_similar_matches.columns.fillna()

Index(['original_index', 'product_type(modified)', 'most_similar_index',
       'most_similar_pre_defined_label'],
      dtype='object')

In [39]:
most_similar_matches.squeeze()

,original_index,product_type(modified),most_similar_index,most_similar_pre_defined_label
0,1,one piece,226.0,one piece
1,3,bikini top,79.0,bikini top
2,4,bikini bottom,78.0,bikini bottom
3,6,bikini top,79.0,bikini top
4,7,bikini bottom,78.0,bikini bottom
...,...,...,...,...
17968,22069,pant,309.0,pant
17969,22070,jewelry,104.0,jewelry
17970,22071,pant,309.0,pant
17971,22072,jewelry,104.0,jewelry


In [40]:
grouped_data.squeeze()

,group_rep_index,group_rep_product_type(modified)
0,0,one piece
1,6109,bikini bottoms
2,6109,bikini bottoms
3,6109,bikini bottoms
4,6109,bikini bottoms
...,...,...
17968,1311,pants
17969,5306,jewelry
17970,1311,pants
17971,5306,jewelry


In [58]:
import numpy as np
np.isnan(most_similar_matches.loc[17972, 'most_similar_index'].item())


True

In [59]:
i = most_similar_matches.at[66, "original_index"]

df.loc[df['original_index'] == i, ['match_most_similar_>80%_string']]


,match_most_similar_>80%_string
66,NaN


In [60]:
#fill up the 80%
empty_cells = 0
for row in range(most_similar_matches.shape[0]):
    index = most_similar_matches.loc[row, "original_index"]
    most_similar_pre_defined_label = most_similar_matches.loc[row, 'most_similar_pre_defined_label']
    most_similar_index = most_similar_matches.loc[row, ['most_similar_index']].item()
    
    if not np.isnan(most_similar_index):
        df.loc[df['original_index'] == index, ['match_most_similar_>80%_string']] = most_similar_pre_defined_label
    else:
        empty_cells += 1

In [150]:
empty_cells

7913

In [61]:
#fill up 60 %
index_similarity_map = {}
index_to_pre_map = {}
for row in range(matches.shape[0]):
    current_index = matches.loc[row, 'right_index']
    label = matches.loc[row, 'left_pre_defined_label']
    similarity = matches.loc[row, 'similarity']
    
    if index_similarity_map.get(current_index) == None:
        index_similarity_map.update({current_index:similarity})
        index_to_pre_map.update({current_index:label})
    elif index_similarity_map.get(current_index) >= similarity:
        index_similarity_map.update({current_index:similarity})
        index_to_pre_map.update({current_index:label})
    else: # similarity < then current, ignore
        continue


In [62]:
for key in index_to_pre_map.keys():
    df.loc[df['current_index'] == key, ['match_string_>60%_similarity']] = index_to_pre_map.get(key)

In [63]:
#fill up group representative of the label 
for row in range(grouped_data.shape[0]):
    df.loc[row, 'group_rep_product_type'] = grouped_data.loc[row, "group_rep_product_type(modified)"]

In [64]:
pair = ("acc", "accessories")
pair[0]

'acc'

In [65]:
#Algorithm 2: if both 60% and 80% don't exist, just ignore, 
#             if the 80% string doesn't exist, compare the 60% string with 80% string similarity, to fill the 80%.
double_check_pairs_pt_word60_dic = {}
#see the accurancy 

def is_nan_string(string):
    return string != string
#w = df.loc[66, ["match_most_similar_>80%_string"]].item()
#print(w)
#is_nan_string(w)
for row in range(df.shape[0]):
    word_80 = df.loc[row, ["match_most_similar_>80%_string"]].item()
    word_60 = df.loc[row, ["match_string_>60%_similarity"]].item()
    
    if is_nan_string(word_80 ) and is_nan_string(word_60):
        continue
    elif is_nan_string(word_80) and not is_nan_string(word_60):
        
        
        pt_modified = df.loc[row, 'product_type(modified)']
        
        pair = (pt_modified, word_60)
        
        if pair not in double_check_pairs_pt_word60_dic.keys():
            double_check_pairs_pt_word60_dic.update({pair: similar(word_60, pt_modified)})
            
        #print("product_type(modified) ", pt_modified, " words(60%): ", word_60, " Similarity: ", similar(word_60, pt_modified))
        if similar(word_60, pt_modified) > 0.33:
            df.loc[row, "match_most_similar_>80%_string"] = word_60

In [66]:
#map most_similar_label to number_of_categories using specific_product_to_num_map
for row in range(df.shape[0]):
    key = df.loc[row, ['match_most_similar_>80%_string']].item()
    if key != None and specific_products_map_to_num.get(key) != None:
        df.loc[row, ['1st_class_label']] = specific_products_map_to_num.get(key)
    else:
        df.loc[row, ['1st_class_label']] = 0
        

In [104]:
main_categories_map_to_num

{'shoes': 1,
 'tops': 2,
 'bottoms': 3,
 'other_clothing': 4,
 'beauty': 5,
 'accessories': 6,
 'homeware': 7,
 'other': 8,
 'unknown': 0}

In [67]:
#(Finally Check √)if the 1st.class.label is unknown to do last check with the algorithm2
for row in range(df.shape[0]):
    key = df.loc[row, ["match_most_similar_>80%_string"]].item()
    if df.loc[row, ['1st_class_label']].item() == 0 and not is_nan_string(key):
        df.loc[row, ['1st_class_label']] = main_categories_map_to_num.get(key)

In [69]:
df.columns

Index(['current_index', 'original_index', 'id', 'title', 'tags', 'body_html',
       'product_type', 'product_type(modified)',
       'match_most_similar_>80%_string', 'match_string_>60%_similarity',
       'group_rep_product_type', '1st_class_label', '2nd_class_label',
       '3rd_class_label', 'raw_text'],
      dtype='object')

In [70]:
df.rename(columns={'1st_class_label':'label_1st', '2nd_class_label':'label_2nd', '3rd_class_label':'label_3rd'}, inplace=True)

In [71]:
unknown = 0
total = df.shape[0]
shoes = 0
other_clothing = 0
tops = 0
bottoms = 0
beauty = 0
home = 0
acc = 0
other = 0
for row in range(df.shape[0]):
    n = df.loc[row, ["label_1st"]].item()
    if n == 0:
        unknown += 1
    elif n == 1:
        shoes += 1
    elif n == 2:
        tops += 1
    elif n == 3:
        bottoms += 1
    elif n == 4:
        other_clothing += 1
    elif n == 5:
        beauty += 1
    elif n == 6:
        acc += 1
    elif n == 7:
        home += 1
    elif n == 8:
        other += 1

In [72]:
totalnum = total

In [73]:
print("unknown", unknown, "\t\tratio of all", unknown/total)
total = totalnum - unknown
print("\nshoes", shoes,"\t\tratio", "{:10.2f}".format(shoes/total))
print("tops", tops, "\t\tratio", "{:10.2f}".format(tops/total))
print("bottoms", bottoms, "\t\tratio", "{:10.2f}".format(bottoms/total))
print("other_clothing", other_clothing,"\tratio", "{:10.2f}".format(other_clothing/total))
print("beauty", beauty, "\t\tratio", "{:10.2f}".format(beauty/total))
print("accessories", acc, "\tratio", "{:10.2f}".format(acc/total))
print("homeware", home, "\t\tratio", "{:10.2f}".format(home/total))
print("other", other, "\t\tratio", "{:10.2f}".format(other/total))


unknown 6194 		ratio of all 0.34462805319089745

shoes 552 		ratio       0.05
tops 3847 		ratio       0.33
bottoms 1681 		ratio       0.14
other_clothing 1633 	ratio       0.14
beauty 526 		ratio       0.04
accessories 3071 	ratio       0.26
homeware 348 		ratio       0.03
other 121 		ratio       0.01


In [74]:
df.columns

Index(['current_index', 'original_index', 'id', 'title', 'tags', 'body_html',
       'product_type', 'product_type(modified)',
       'match_most_similar_>80%_string', 'match_string_>60%_similarity',
       'group_rep_product_type', 'label_1st', 'label_2nd', 'label_3rd',
       'raw_text'],
      dtype='object')

In [75]:
#df.rename(columns={"1st_class_label": "label_1st", "2nd_class_label": "label_2nd", "3rd_class_label": "label_3rd)"}, inplace=True)

k = df.loc[df.label_1st == 0].index
del(df['group_rep_product_type'])
df = df.drop(k)

In [178]:
#June 17 2022
#df.to_csv("(temp)first_class_label_done.csv", index=False)


In [111]:
print("blanket and blanket/towel: ", similar("blanket", "blanket/towel"))
print("accessory and accessories: ", similar("accessory", "accessories"))
print("top and tops", similar("tops", "top"))
print("tops and tops-casual", similar("tops", "tops-casual"))
print("denim and denim skirt", similar("denim", "denim skirt"))
print("dress and dresses", similar("dress", "dresses"))
print("accessories and accessories-scarves", similar('accessories', "accessories-scarves"))
#todo: 



blanket and blanket/towel:  0.7
accessory and accessories:  0.8
top and tops 0.8571428571428571
tops and tops-casual 0.5333333333333333
denim and denim skirt 0.625
dress and dresses 0.8333333333333334
accessories and accessories-scarves 0.7333333333333333


In [94]:
unknown_items = {}
for i in range(df.shape[0]):
    num = df.loc[i, "label_1st"] 
    if num != 0:
        continue
    key = df.loc[i, "product_type"]
    if key not in unknown_items.keys():
        unknown_items.update({key: 1})
    else:
        assert unknown_items.get(key) != None
        new_val = unknown_items.get(key) + 1
        unknown_items.update({key:new_val})
        

In [95]:
all_cat.__contains__("lip")

True

In [96]:
unknown_items = dict(sorted(unknown_items.items(), key=lambda item:item[1], reverse=True))
unknown_items

{}

In [92]:
double_check_pairs_pt_word60_dic

{('underwear - underwear - boxer briefs - cotton',
  'underwear'): 0.3333333333333333,
 ('underwear - underwear - boxer briefs - tech',
  'underwear'): 0.34615384615384615,
 ('headwear - headwear - hats - mesh back', 'headwear'): 0.34782608695652173,
 ('dresses - sun dresses', 'dress'): 0.38461538461538464,
 ('headwear - headwear - beanies', 'headwear'): 0.43243243243243246,
 ('tankini bikini tops', 'tankini'): 0.5384615384615384,
 ('bottoms - trousers', 'trouser'): 0.56,
 ('bralette bikini tops', 'bralette'): 0.5714285714285714,
 ('parka jacket', 'parka'): 0.5882352941176471,
 ('swim wear - bottoms', 'swimwear'): 0.5925925925925926,
 ('accessories hair care', 'hair care'): 0.6,
 ('swim wear - one piece', 'one piece'): 0.6,
 ('tees', 'tee shirt'): 0.6153846153846154,
 ('lip balm', 'lip balm treatment'): 0.6153846153846154,
 ('robe', 'bath robe'): 0.6153846153846154,
 ('flat', 'flat shoe'): 0.6153846153846154,
 ('swim wear - bottom', 'swimwear'): 0.6153846153846154,
 ('denim', 'denim sh

In [80]:
double_check_pairs_pt_word60_dic = dict(sorted(double_check_pairs_pt_word60_dic.items(), key=lambda item: item[1]))

In [81]:
double_check_pairs_pt_word60_dic

{('underwear - underwear - boxer briefs - cotton',
  'underwear'): 0.3333333333333333,
 ('underwear - underwear - boxer briefs - tech',
  'underwear'): 0.34615384615384615,
 ('headwear - headwear - hats - mesh back', 'headwear'): 0.34782608695652173,
 ('dresses - sun dresses', 'dress'): 0.38461538461538464,
 ('headwear - headwear - beanies', 'headwear'): 0.43243243243243246,
 ('tankini bikini tops', 'tankini'): 0.5384615384615384,
 ('bottoms - trousers', 'trouser'): 0.56,
 ('bralette bikini tops', 'bralette'): 0.5714285714285714,
 ('parka jacket', 'parka'): 0.5882352941176471,
 ('swim wear - bottoms', 'swimwear'): 0.5925925925925926,
 ('accessories hair care', 'hair care'): 0.6,
 ('swim wear - one piece', 'one piece'): 0.6,
 ('tees', 'tee shirt'): 0.6153846153846154,
 ('lip balm', 'lip balm treatment'): 0.6153846153846154,
 ('robe', 'bath robe'): 0.6153846153846154,
 ('flat', 'flat shoe'): 0.6153846153846154,
 ('swim wear - bottom', 'swimwear'): 0.6153846153846154,
 ('denim', 'denim sh

In [82]:
df = pd.read_csv("(temp)first_class_label_done.csv")
original_df = pd.read_csv("/Users/luis/Downloads/products.csv").reset_index()
df.insert(df.columns.get_loc('raw_text'), "gender", 0, allow_duplicates=True)
df.insert(df.columns.get_loc('raw_text'), "bucket_number", 0, allow_duplicates=True)
df.insert(df.columns.get_loc('raw_text'), "buckets", "", allow_duplicates=True)
df.insert(df.columns.get_loc('raw_text'), "color", 0, allow_duplicates=True)
df.insert(df.columns.get_loc('raw_text'), "url", "", allow_duplicates=True)
df.insert(df.columns.get_loc('raw_text'), "patterns", "", allow_duplicates=True)


In [83]:
original_df.columns

Index(['index', 'id', 'title', 'handle', 'vendor', 'tags', 'variants',
       'images', 'options', 'rating', 'colors', 'sizes', 'buckets',
       'body_html', 'business_name', 'compare_at_price', 'created_at',
       'original_price', 'product_type', 'published_at', 'updated_at',
       'is_on_sale', 'sale_ratio', 'place_id', 'url', 'gender', 'is_available',
       'sh_id'],
      dtype='object')

In [84]:
df.columns

Index(['current_index', 'original_index', 'id', 'title', 'tags',
       'product_type', 'product_type(modified)',
       'match_most_similar_>80%_string', 'match_string_>60%_similarity',
       'label_1st', 'label_2nd', 'label_3rd)', 'gender', 'bucket_number',
       'buckets', 'color', 'url', 'patterns', 'raw_text'],
      dtype='object')

In [85]:
id = df.loc[0, 'original_index']

type(original_df.loc[original_df["index"] == id, "url"].item())
df.loc[0, 'url']

''

In [86]:
#build our cleaned dataset with some attributes required
for row in range(df.shape[0]):
    index = df.loc[row, 'original_index']
    df.loc[row, 'url'] = original_df.loc[original_df["index"] == index, "url"].item()
    df.loc[row, "buckets"] = original_df.loc[original_df["index"] == index, "buckets"].item().removeprefix("{").removesuffix("}")
    

In [251]:
#df.rename(columns={"label_3rd)": "label_3rd"}, inplace=True)

In [87]:
#new addition June 21
import pandas as pd
df = pd.read_csv("(Current)cleaned_dataset_v1.csv")
del(df["Unnamed: 0"])
original_df = pd.read_csv("/Users/luis/Documents/MyUA/2022Summer/products.csv").reset_index()
df.insert(df.columns.get_loc("product_type"), "images", "", allow_duplicates=True)
df.insert(df.columns.get_loc("raw_text") ,"body_html", "", allow_duplicates=True)
for row in range(df.shape[0]):
    index = df.loc[row, 'original_index']
    df.loc[row, 'images'] = original_df.loc[original_df["index"] == index, "images"].item()
    df.loc[row, 'body_html'] = original_df.loc[original_df["index"] == index, "body_html"].item()

In [89]:
#df.to_csv("(v1.3)cleaned_dataset.csv", index=False)
